# Wisconsin Scraping 

In [10]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from io import BytesIO
from urllib.request import urlopen

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [4]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [5]:
# state - UPDATE EACH TIME
state = 'wisconsin'

In [ ]:
# review of robots.txt - non existent

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape links 
`Selenium`

In [ ]:
driver = webdriver.Chrome()
driver.get('https://evers.wi.gov/Pages/Newsroom/Press-Releases.aspx')

wait = WebDriverWait(driver, 3)

wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="heading2023"]/a')))

apply = driver.find_element(By.XPATH, '//*[@id="heading2023"]/a')

driver.execute_script("arguments[0].click();", apply);

all_2023 = driver.find_element(By.XPATH, '//*[@id="collapse2023"]/div')

links = all_2023.find_elements(By.TAG_NAME, 'li')

results_list = [{'title': i.find_element(By.TAG_NAME, 'a').text,
                'date': i.text, 
                'link': i.find_element(By.TAG_NAME, 'a').get_attribute('href')} for i in links]

df = pd.DataFrame.from_dict(results_list)
df.title = [i.split('—')[0].strip() for i in df.title]
df.date = [i.split('—')[1].strip() for i in df.date]
    
driver.close()

In [ ]:
df.head()

In [ ]:
# add empty content column 
df['content'] = np.nan
df.head()

In [ ]:
# export
df.to_csv(f'{state}_links.csv', index = False)

In [6]:
test = pd.read_csv(f'{state}_links.csv')
test.head()

,title,date,link,content
0,Gov. Evers Orders Flags to Half-Staff in Honor...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN
1,2023 Roundup: Gov. Evers Completes Annual 72-C...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN
2,Gov. Evers Seeks Applicants for Langlade Count...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN
3,Gov. Evers Releases Statement Regarding Passin...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN
4,"Gov. Evers, First Lady Celebrate Kwanzaa in Wi...","December 26, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN


## Scrape Content 
`Selenium`

### Test

In [12]:
def scrape_wisconsin_content(df, link):
    window_width = 50
    window_height = 50
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f"--window-size={window_width},{window_height}")
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else:
            try: 
                driver = webdriver.Chrome(options=chrome_options)
#                 driver = webdriver.Chrome()
                driver.get(link)
                time.sleep(3)

                results = driver.find_element(By.XPATH, '//*[@id="bulletin_body"]/table/tbody/tr[7]/td')

                final_text = results.text 
                final_text = final_text.replace(u'\n', ' ')
                if len(final_text) < 200: 
                    try: 
                        results = driver.find_element(By.XPATH, '//*[@id="bulletin_body"]/table/tbody/tr[8]/td')
                        final_text = results.text 
                        final_text = final_text.replace(u'\n', ' ')
                        df.loc[df.link == link, 'content'] = final_text
                        return f"success: content added from {link}"
                    except: 
                        pass
                driver.close()

                df.loc[df.link == link, 'content'] = final_text
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"  

In [ ]:
combined = df.copy()

In [14]:
for i in combined.link[0:10]:
    scrape_wisconsin_content(combined, i)

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3821c31'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/381fb42'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/381dfd4'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3820ce5'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/38046d6'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3816300'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/381d86b'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3816d6c'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3809598'

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/380fb39'

In [15]:
combined.head()

,title,date,link,content
0,Gov. Evers Orders Flags to Half-Staff in Honor...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today ordered the fl...
1,2023 Roundup: Gov. Evers Completes Annual 72-C...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today announced that...
2,Gov. Evers Seeks Applicants for Langlade Count...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers announced today that...
3,Gov. Evers Releases Statement Regarding Passin...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today released a sta...
4,"Gov. Evers, First Lady Celebrate Kwanzaa in Wi...","December 26, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers and First Lady Kathy...


### Scrape content from all links

In [16]:
for i in combined.link:
    scrape_wisconsin_content(combined, i)

'content already existing for https://content.govdelivery.com/accounts/WIGOV/bulletins/3821c31'

'content already existing for https://content.govdelivery.com/accounts/WIGOV/bulletins/381fb42'

'content already existing for https://content.govdelivery.com/accounts/WIGOV/bulletins/381dfd4'

'content already existing for https://content.govdelivery.com/accounts/WIGOV/bulletins/3820ce5'

'content already existing for https://content.govdelivery.com/accounts/WIGOV/bulletins/38046d6'

...

'success: content added from https://lnks.gd/2/zLx_6x'

'success: content added from https://lnks.gd/2/zJNnN5'

'success: content added from https://lnks.gd/2/zLbRD6'

'success: content added from https://lnks.gd/2/zJPXVt'

'success: content added from https://lnks.gd/2/zHdjZX'

In [19]:
# check
combined.info()
combined.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    364 non-null    object
 1   date     364 non-null    object
 2   link     364 non-null    object
 3   content  360 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


,title,date,link,content
0,Gov. Evers Orders Flags to Half-Staff in Honor...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today ordered the fl...
1,2023 Roundup: Gov. Evers Completes Annual 72-C...,"December 28, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today announced that...
2,Gov. Evers Seeks Applicants for Langlade Count...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers announced today that...
3,Gov. Evers Releases Statement Regarding Passin...,"December 27, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today released a sta...
4,"Gov. Evers, First Lady Celebrate Kwanzaa in Wi...","December 26, 2023",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers and First Lady Kathy...


## Validate and Clean

In [26]:
# review null content
nas = combined[combined.content.isna()]

for link in nas.link:
    link
    
# looks 3 have an extra https and 1 is repeated in a single string 

'http://https//content.govdelivery.com/accounts/WIGOV/bulletins/357ae6b'

'http://https//content.govdelivery.com/accounts/WIGOV/bulletins/354196d'

'https://content.govdelivery.com/accounts/WIGOV/bulletins/3517f17https://content.govdelivery.com/accounts/WIGOV/bulletins/3517f17'

'http://https//content.govdelivery.com/accounts/WIGOV/bulletins/34a8199'

In [38]:
# print indices
nas

,title,date,link,content
239,Gov. Evers Appoints Clifford Burdon as Marquet...,"May 1, 2023",http://https//content.govdelivery.com/accounts...,NaN
249,"Gov. Evers, DATCP Announce Commercial Nitrogen...","April 20, 2023",http://https//content.govdelivery.com/accounts...,NaN
276,Gov. Evers Celebrates Launch of New Clean Econ...,"March 29, 2023",https://content.govdelivery.com/accounts/WIGOV...,NaN
305,Gov. Evers Orders Flags to Half-Staff in Honor...,"February 24, 2023",http://https//content.govdelivery.com/accounts...,NaN


In [39]:
#hard code replacements
combined.link[239] = 'https://content.govdelivery.com/accounts/WIGOV/bulletins/357ae6b'

title      Gov. Evers Appoints Clifford Burdon as Marquet...
date                                             May 1, 2023
link       https://content.govdelivery.com/accounts/WIGOV...
content                                                  NaN
Name: 239, dtype: object

In [50]:
combined.link[249] 
combined.link[249] = 'https://content.govdelivery.com/accounts/WIGOV/bulletins/354196d'

'https://content.govdelivery.com/accounts/WIGOV/bulletins/354196d'

In [54]:
combined.link[305] 
combined.link[305] = 'https://content.govdelivery.com/accounts/WIGOV/bulletins/34a8199'

'https://content.govdelivery.com/accounts/WIGOV/bulletins/34a8199'

In [69]:
combined.link[276]
combined.link[276] = 'https://content.govdelivery.com/accounts/WIGOV/bulletins/3517f17'

'https://content.govdelivery.com/accounts/WIGOV/bulletins/3517f17'

In [40]:
# redo 
scrape_wisconsin_content(combined, combined.link[239])

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/357ae6b'

In [60]:
# check this worked
combined.content[239]

"MADISON — Gov. Tony Evers announced today his appointment of Clifford Burdon to serve as Marquette County District Attorney. The appointment fills a vacancy created by former District Attorney Brian J. Juech’s resignation. Burdon will serve for the remainder of the unexpired term that ends January 2025.  “Clifford Burdon brings experience and continuity to the Marquette County District Attorney’s Office,” said Gov. Evers. “He will serve the people of Marquette County well as their next district attorney.”  Burdon is currently the Assistant District Attorney in the Marquette County District Attorney’s Office and has been a prosecutor for more than a decade, previously working in the Juneau, Fond du Lac, and Columbia County District Attorney Offices. Prior to his current role and earlier in his career, Burdon was a criminal defense attorney. Over his career as a prosecutor and defense attorney, he has handled various criminal matters and dozens of jury trials.  “Governor Evers has made 

In [59]:
# redo 
scrape_wisconsin_content(combined, combined.link[249])

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/354196d'

In [65]:
# success
combined.content[249]

'MADISON — Gov. Tony Evers, together with the Wisconsin Department of Agriculture, Trade, and Consumer Protection (DATCP), announced that 20 recipients are receiving grant funding for the 2023 Commercial Nitrogen Optimization Pilot Program (NOPP). These grants aim to refine and enhance the understanding of new methods that optimize commercial nitrogen applied to agricultural fields, helping to protect vital soil and water resources.  “This pilot program is a great way to bring together partners in this work, from farmers and producers to conservation professionals, to promote collaboration and find best practices to ensure that we are protecting our state’s natural resources from harmful contaminants while helping farmers protect their bottom line,” said Gov. Evers. “Farmers know firsthand the value of having clean water and fruitful land, and we are glad to continue to invest in and support producer-led conservation efforts in my 2023-25 biennial budget so farmers can continue to stay

In [61]:
# retry
scrape_wisconsin_content(combined, combined.link[305])

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/34a8199'

In [62]:
# success 
combined.content[305]

'MADISON — Gov. Tony Evers today ordered the flags of the United States and the state of Wisconsin to be flown at half-staff on Sun., Feb. 26, 2023, in honor of Fire Chief Todd J. Yandre of the Lake Mills Fire Department, who passed away in the line of duty on Feb. 15, 2023.    “Chief Yandre was by all accounts a caring and loyal friend, mentor, coach, husband, father, and grandfather who was dedicated to serving his community. Chief Yandre served the Lake Mills Fire Department for more than four decades, first as a volunteer firefighter and eventually becoming chief of the department,” said Gov. Evers. “Kathy and I share our deepest condolences with the Yandre family, his wife, Monica, his children, Jordan and Catherine, his grandchildren, and all of his friends and family at the Lake Mills Fire Department. He will not be forgotten.”  Chief Yandre graduated from Lake Mills High School and later earned his degree in emergency management with an emphasis in fire safety from Lakeshore Te

In [70]:
# retry
scrape_wisconsin_content(combined, combined.link[276])

'success: content added from https://content.govdelivery.com/accounts/WIGOV/bulletins/3517f17'

In [71]:
# success
combined.content[276]

'MADISON — Gov. Tony Evers today joined a broad-based group of climate, energy, environmental justice, and conservation organizations, businesses, and allies in Milwaukee to celebrate the official launch of the new Clean Economy Coalition of Wisconsin (CECW).  “Today, we have an historic opportunity to define Wisconsin as a clean economy state—to build a workforce, economy, and state that’s prepared for the 21st century, reduce reliance on out-of-state energy sources and lower energy bills for working families, and create new jobs and apprenticeships in innovative industries,” said Gov. Evers. “Now is the time for us to double our efforts to build the future we want for our state, and I’m so excited for the role the Clean Economy Coalition of Wisconsin is going to play in our efforts to do just that.”  The CECW seeks to leverage the combined strength of its members to enhance the quality of life of all Wisconsinites by accelerating the state’s transition to a clean energy economy. Foun

In [72]:
# double check string length 

filtered_rows = []

for index, row in combined.iterrows():
    if pd.isna(row['content']):
        pass  
    elif len(row['content']) < 200:
        filtered_rows.append(row)
    else:
        pass
    
redo = pd.DataFrame(filtered_rows)

In [73]:
# empty so success
redo

""


In [74]:
# clean date column 
combined['date_clean'] = pd.to_datetime(combined['date'])

combined_2023 = combined.drop(columns=['date']).copy()

In [76]:
combined_2023.info()
combined_2023.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       364 non-null    object        
 1   link        364 non-null    object        
 2   content     364 non-null    object        
 3   date_clean  364 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 11.5+ KB


,title,link,content,date_clean
0,Gov. Evers Orders Flags to Half-Staff in Honor...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today ordered the fl...,2023-12-28
1,2023 Roundup: Gov. Evers Completes Annual 72-C...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today announced that...,2023-12-28
2,Gov. Evers Seeks Applicants for Langlade Count...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers announced today that...,2023-12-27
3,Gov. Evers Releases Statement Regarding Passin...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today released a sta...,2023-12-27
4,"Gov. Evers, First Lady Celebrate Kwanzaa in Wi...",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers and First Lady Kathy...,2023-12-26


## Export

In [77]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 364 documents from wisconsin for 2023


In [78]:
# export
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [79]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,content,date_clean
0,Gov. Evers Orders Flags to Half-Staff in Honor...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today ordered the fl...,2023-12-28
1,2023 Roundup: Gov. Evers Completes Annual 72-C...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today announced that...,2023-12-28
2,Gov. Evers Seeks Applicants for Langlade Count...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers announced today that...,2023-12-27
3,Gov. Evers Releases Statement Regarding Passin...,https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers today released a sta...,2023-12-27
4,"Gov. Evers, First Lady Celebrate Kwanzaa in Wi...",https://content.govdelivery.com/accounts/WIGOV...,MADISON — Gov. Tony Evers and First Lady Kathy...,2023-12-26
